In [ ]:
map = [[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0]]
start = (0,0)
goal = (5,4)

def dist(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def aStar(map, start, goal):
    seen = [start]
    visited = []
    while seen:
        pos = seen.pop(0)
        visited.append(pos)
        if pos == goal:
            return visited
        x = -1
        y = -1
        neighbors = []
        min_d = dist(pos, goal)
        next = None
        while y < 2:
            x = -1
            while x < 2:
                yp = pos[1]+y
                xp = pos[0]+x
                if yp < len(map) and xp < len(map[0]) and (y != 0 or x != 0) and (yp >= 0 and xp >=0) and (xp,yp) not in seen and (xp,yp) not in visited and map[yp][xp] == 0:
                    neighbors.append((xp,yp))
                x += 1
            y += 1
        for n in neighbors:
            nd = dist(n, goal)
            if nd < min_d:
                min_d = nd
                next = n
        seen.append(n)


aStar(map, start, goal)

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 2),
 (3, 1),
 (2, 1),
 (3, 0),
 (4, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4)]